<a href="https://colab.research.google.com/github/junsookim76/Ewha_2023/blob/main/8_Regression_House.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 15장 실제 데이터로 만들어 보는 모델

## 1. 데이터 파악하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

# 깃허브에 준비된 데이터를 가져옵니다.
!git clone https://github.com/junsookim76/Data.git

# 집 값 데이터를 불러옵니다.
df = pd.read_csv("./Data/house_train_selected.csv")

# 데이터를 미리 살펴보겠습니다. 
df

fatal: destination path 'Data' already exists and is not an empty directory.


,Unnamed: 0,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,BsmtQual_Ex,TotRmsAbvGrd,SalePrice
0,0,7,1710,2,548,856,856,2,0,8,208500
1,1,6,1262,2,460,1262,1262,2,0,6,181500
2,2,7,1786,2,608,920,920,2,0,6,223500
3,3,7,1717,3,642,756,961,1,0,7,140000
4,4,8,2198,3,836,1145,1145,2,0,9,250000
...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,6,1647,2,460,953,953,2,0,7,175000
1456,1456,6,2073,2,500,1542,2073,2,0,7,210000
1457,1457,7,2340,1,252,1152,1188,2,0,9,266500
1458,1458,5,1078,1,240,1078,1078,1,0,5,142125


## 2. 주택 가격 예측 모델

In [ ]:
# 집 값을 제외한 나머지 열을 저장합니다. 
X_train_pre = df.drop(columns=['SalePrice'])

# 집 값을 저장합니다.
y = df['SalePrice'].values


ERROR! Session/line number was not unique in database. History logging moved to new session 183


In [ ]:
# 전체의 80%를 학습셋으로, 20%를 테스트셋으로 지정합니다.
X_train, X_test, y_train, y_test = train_test_split(X_train_pre, y, test_size=0.2)

In [ ]:
# 모델의 구조를 설정합니다.
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(1))
model.summary()

# 모델을 실행합니다.
model.compile(optimizer ='adam', loss = 'mean_squared_error')

# 20회 이상 결과가 향상되지 않으면 자동으로 중단되게끔 합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델의 이름을 정합니다.
modelpath="./Data/model/house.hdf5"

# 최적화 모델을 업데이트하고 저장합니다.
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

# 실행 관련 설정을 하는 부분입니다. 전체의 20%를 검증셋으로 설정합니다. 
history = model.fit(X_train, y_train, validation_split=0.25, epochs=2000, batch_size=32, callbacks=[early_stopping_callback, checkpointer])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jskim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/5j/5vnwyr3s4vjb2zhwj5m319br0000gn/T/ipykernel_25199/3838399813.py", line 3, in <module>
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jskim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jskim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    ret

In [ ]:
# 예측 값과 실제 값, 실행 번호가 들어갈 빈 리스트를 만듭니다.
real_prices =[]
pred_prices = []
X_num = []

# 25개의 샘플을 뽑아 실제 값, 예측 값을 출력해 봅니다. 
n_iter = 0
Y_prediction = model.predict(X_test).flatten()
for i in range(25):
    real = y_test[i]
    prediction = Y_prediction[i]
    print("실제가격: {:.2f}, 예상가격: {:.2f}".format(real, prediction))
    real_prices.append(real)
    pred_prices.append(prediction)
    n_iter = n_iter + 1
    X_num.append(n_iter)

In [ ]:
# 그래프를 통해 샘플로 뽑은 25개의 값을 비교해 봅니다.

plt.plot(X_num, pred_prices, label='predicted price')
plt.plot(X_num, real_prices, label='real price')
plt.legend()
plt.show()